# cbvLang
cbvEval 노트북에서 정의했던 내용을 그대로 가져와서
Tm이라는 타입의 이름을 Expr로 바꾸고
거기에 몇 가지 프로그래밍 요소를 추가했다.

In [2]:
-- 변수 이름은 문자열로 나타낸다
type Nm = String

-- 장난감 함수형 언어 = 람다식 + ...
data Expr
  = Var Nm          -- x, y, z, ...
  | I Int           -- 0, -1, 1, -2, 2, -3, 3, ...
  | B Bool          -- False, True
  | Lam Nm Expr      -- \x.e
  | Rec Nm Expr      -- rec f e
  | App Expr Expr      -- e1 e2
  | If Expr Expr Expr  -- if e then e1 else e2
  | Let Nm Expr Expr  -- let x=e2 in e
  | Add Expr Expr     -- e1 + e2
  | Mul Expr Expr     -- e1 * e2
  | Neg Expr         -- - e
  | Eq Expr Expr      -- e1 == e2
  | Ne Expr Expr      -- e1 /= e2
  | Lt Expr Expr      -- e1 <  e2
  | Gt Expr Expr      -- e1 >  e2
  | Le Expr Expr      -- e1 <= e2
  | Ge Expr Expr      -- e1 >= e2
  | And Expr Expr     -- e1 && e2
  | Or Expr Expr      -- e1 || e2
  | Not Expr         -- not e
  deriving (Show, Eq)

In [47]:
import IHaskell.Display

dispExpr t = Display [html("<code>"++dpExpr t++"</code>")]

dpt t@(Lam{}) = paren (dpExpr t)
dpt t@(Let{}) = paren (dpExpr t)
dpt t@(If{})  = paren (dpExpr t)
dpt t@(Rec{}) = paren (dpExpr t)
dpt t         = dpExpr t

dps s@(I _)   = dpExpr s
dps s@(B _)   = dpExpr s
dps s@(Var{}) = dpExpr s
dps s         = paren (dpExpr s)

dpe = dps

dpExpr :: Expr -> String
dpExpr (Var x) = x
dpExpr (Lam x t) = "λ" ++ x ++ "." ++ dpExpr t
dpExpr (App t s) = dpt t ++ " " ++ dps s
dpExpr (I n) = show n
dpExpr (B b) = show b
dpExpr (Add e1 e2) = dpe e1++" + "++dpe e2
dpExpr (Mul e1 e2) = dpe e1++" * "++dpe e2
dpExpr (Neg e) = "-"++dpe e
dpExpr (Eq e1 e2) = dpe e1++" == "++dpe e2
dpExpr (Ne e1 e2) = dpe e1++" /= "++dpe e2
dpExpr (Lt e1 e2) = dpe e1++" < "++dpe e2
dpExpr (Gt e1 e2) = dpe e1++" > "++dpe e2
dpExpr (Le e1 e2) = dpe e1++" <= "++dpe e2
dpExpr (Ge e1 e2) = dpe e1++" >= "++dpe e2
dpExpr (And e1 e2) = dpe e1++" && "++dpe e2
dpExpr (Or e1 e2) = dpe e1++" || "++dpe e2
dpExpr (Not e) = "not "++ dpe e
dpExpr (Let x e2 e) = "let "++x++" = "++dpt e2++" in "++dpExpr e
dpExpr (Rec f e) = "rec "++f++" "++dpt e
dpExpr (If e e1 e2) = "if "++dpe e++" then "++dpe e1++" else "++dpe e2

paren s = "(" ++ s ++ ")"

In [48]:
x = Var "x"
y = Var "y"

x
y

:type dpExpr
putStr (dpExpr x)
putStr $ dpExpr y

Var "x"

Var "y"

dpExpr :: Expr -> String

x

y

In [49]:
I 3 -- 상수 3
putStr $ dpExpr (I 3)
B True -- 참 상수값
putStr $ dpExpr (B True)

I 3

3

B True

True

In [50]:
-- \x.x
:type Lam
Lam "x" (Var "x") 
putStr $ dpExpr (Lam "x" (Var "x"))
putStr . dpExpr $ Lam "x" (Var "x")

Lam :: Nm -> Expr -> Expr

Lam "x" (Var "x")

λx.x

λx.x

In [51]:
-- \f.(\x.f(f x))
-- 하스켈: \f -> (\x -> f (f x))
-- 우리 장난감 언어의 문법구조 Expr로는
:type App
Lam "f" (Lam "x" (App (Var "f") (App (Var "f") (Var "x"))))
it -- 바로 직전값
putStr (dpExpr it)

App :: Expr -> Expr -> Expr

Lam "f" (Lam "x" (App (Var "f") (App (Var "f") (Var "x"))))

Lam "f" (Lam "x" (App (Var "f") (App (Var "f") (Var "x"))))

λf.λx.f (f x)

In [52]:
-- (\x -> if x>=0 then x else -x) -- 하스켈로 절대값 함수
:type If
x = Var "x"
Lam "x" (If (x `Ge` I 0) x (Neg x))
putStr (dpExpr it)

If :: Expr -> Expr -> Expr -> Expr

Lam "x" (If (Ge (Var "x") (I 0)) (Var "x") (Neg (Var "x")))

λx.if (x >= 0) then x else (-x)

In [53]:
idExpr = Lam "x" (Var "x")

putStrLn $ dpExpr $ Lam "y" (App idExpr (App idExpr (Var "y")))

dispExpr $ Lam "y" (App idExpr (App idExpr (Var "y")))

λy.(λx.x) ((λx.x) y)

λy.(λx.x) ((λx.x) y)

In [54]:
putStrLn $ dpExpr $ I 3 `Add` I 4
putStrLn $ dpExpr $ Let "id" idExpr (App (Var "id") (Var "id"))

3 + 4

let id = (λx.x) in id id

In [56]:
-- eval을 완성한 후 factExpr를 완성하여 7팩토리알을 실행해 보라
:type Rec

x = Var "x"
f = Var "f"

sub :: Expr -> Expr -> Expr
sub e1 e2 = Add e1 (Neg e2)

factExpr = Rec "f"
              (Lam "x"
                   (If (Gt x (I 1))
                       (I 1)
                         (Mul x (App f (sub x (I 1))))))
--                       (Mul x (App f (Add x (Neg (I 1))))))
expr = Let "fact" factExpr $ Var "fact" `App` I 3 
-- eval expr []
putStr (dpExpr factExpr)
putStr (dpExpr expr)

Rec :: Nm -> Expr -> Expr

rec f (λx.if (x > 1) then 1 else (x * (f (x + (-1)))))

let fact = (rec f (λx.if (x > 1) then 1 else (x * (f (x + (-1)))))) in fact 3

# 장난감 함수형 언어의 call-by-value evaluation
call-by-value evaluation (줄여서 CVB evaluation)은
적극적 계산법(eager evaluation)이라고 부르기도 한다.

람다식에 대한 적극적 계산법의 big-step 실행의미를 추론규칙 형태로 정의했던 규칙에 덧붙여
새로 장난감 함수형 언어에 추가된 요소들에 대한 규칙들 중요한 것들을 추려서 제시하였다.

$\displaystyle\begin{array}{rcrcl}
\sigma&\in&\textit{Env} &=&
\textit{Nm} \longrightarrow^{\hspace{-2.7ex}\textrm{fin}} \textit{Val} \\
v&\in&\textit{Val} &=&  \textit{Int} ~+~ \textit{Bool} ~+~ \textit{Expr}_V \times Env \\
n&\in&\textit{Int}&=& \{\ldots,-2,-1,0,1,2,\ldots\} \\
b&\in&\textit{Bool}&=& \{\textbf{False},\textbf{True}\} \\
\end{array}$

$\displaystyle\begin{array}{rcl}
\textit{Expr}_V &\subset& Expr\\
\textit{Expr}_V &=& \{ \lambda x.e \mid x\in\textit{Nm}, e\in\textit{Expr}\}
              \cup \{ \textbf{rec}~f~\lambda x.e \mid f\in\textit{Nm}, x\in\textit{Nm}, e\in\textit{Expr}\}
\end{array}$

$~$
$~$

$\displaystyle\Downarrow ~:~ \textit{Expr}\times \textit{Env} \to \textit{Val} $

$~$

$\displaystyle\frac{}{(x, \sigma) \Downarrow \sigma(x)} 
\quad
 \displaystyle\frac{}{(\lambda x.e, \sigma) \Downarrow \langle\lambda x.e, \sigma\rangle}
\quad
 \displaystyle\frac{}{(\textbf{rec}~f~\lambda x.e, \sigma) \Downarrow \langle\textbf{rec}~f~\lambda x.e, \sigma\rangle}
$

$~$

$
\displaystyle\frac{}{(n, \sigma) \Downarrow n}
\qquad
\displaystyle\frac{}{(\textbf{True}, \sigma) \Downarrow \textbf{True}}
\quad
\displaystyle\frac{}{(\textbf{False}, \sigma) \Downarrow \textbf{False}}
$

$~$

$\displaystyle\frac{
   ((\lambda x.e)~e_2, \sigma) \Downarrow v}{
   (\textbf{let}~x=e_2~e, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow \langle\lambda x.e, \sigma_1\rangle \\
   (e_2,\sigma) \Downarrow v_2 \\
   (e, \{x\mapsto v_2\}\sigma_1) \Downarrow v
   \end{array}~}{
   (e_1~e_2, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow \langle\textbf{rec}~f~\lambda x.e, \sigma_1\rangle \\
   (e_2,\sigma) \Downarrow v_2 \\
   (e, \{x\mapsto v_2,\;f\mapsto\langle\textbf{rec}~f~\lambda x.e,\sigma_1\rangle\}\sigma_1) \Downarrow v
   \end{array}~}{
   (e_1~e_2, \sigma) \Downarrow v}
$

$~$


$\displaystyle\frac{~
   (e,\sigma) \Downarrow \textbf{True} \quad
   (e_1,\sigma) \Downarrow v}{
   (\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2, \sigma) \Downarrow v}
\qquad
 \displaystyle\frac{~
   (e,\sigma) \Downarrow \textbf{False} \quad
   (e_2,\sigma) \Downarrow v}{
   (\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2, \sigma) \Downarrow v}
$

$~$

$\displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow n_1\\
   (e_2,\sigma) \Downarrow n_2\\
   n = n_1 \stackrel{\textit{Int}}{+} n_2
   \end{array}~}{
   (e_1 + e_2, \sigma) \Downarrow n}
\qquad\cdots\quad\cdots
\qquad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow b_1\\
   (e_2,\sigma) \Downarrow b_2\\
   b = b_1 \land b_2
   \end{array}~}{
   (b_1 \mathop{\mathsf{\&\!\!\&}\,} b_2, \sigma) \Downarrow b}
\qquad\cdots\quad\cdots
$

$~$

$
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow v_1\\
   (e_2,\sigma) \Downarrow v_2\\
   v_1 \stackrel{\tau}{=} v_2
   \end{array}~}{
   (e_1 = e_2, \sigma) \Downarrow \textbf{True}}~~(\tau\in\{\textit{Int},\textit{Bool}\})
\quad
 \displaystyle\frac{~
   \begin{array}{l}
   (e_1,\sigma) \Downarrow v_1\\
   (e_2,\sigma) \Downarrow v_2\\
   v_1 \stackrel{\tau}{\neq} v_2
   \end{array}~}{
   (e_1 = e_2, \sigma) \Downarrow \textbf{False}}~~(\tau\in\{\textit{Int},\textit{Bool}\})
\qquad\cdots\quad\cdots
$

위의 표기법은 비유하자면 $v = f(x)$라고 쓰는 대신에 $f(x)$를 기계적인 규칙을 따라 계속 계산을 돌리다 보면 $v$라는 값에 도달한다는 의미로 $f(x) \Downarrow v$라는 식으로 표시하는 것이다. 그리고 바로 람다식의 적극적 계산법을 위한 기계적인 규칙이 위에 나와 있는 세 가지 규칙이다.
산술연산은 Int 타입의 정수값에 대해서만, 논리연산은 Bool 타입의 진리값에 대해서만, 비교연산은 Int 타입의 정수값끼리 또는 Bool 타입의 진리값끼리만 정의되어 있다. 여기서는 논리연산의 short circuit에 대해서는 일단 생각하지 않기로 하자.

In [6]:
-- 1부터 넘겨받은 값까지 합을 구하는 함수
--- rec f \x -> if x<1 then 0 else x + f (x-1)

In [7]:
-- finite mapping을 순서쌍 리스트 타입으로 정의
type Env = [(Nm, Val)]
-- Expr_V를 따로 정의하지 않고 그냥 Expr을 이용해 정의
data Val = VI Int | VB Bool | Cl Expr Env  deriving Show

-- simga(x)에 해당하는 것이 lookup x simga
lookup' x env  = v  where Just v = lookup x env

In [8]:
f 0 = 0
f 1 = 3
f n = 2 * n

f' x = case x of
         0 -> 0
         1 -> 3
         n -> 2 * n

In [9]:
f 0
f' 0
f 1
f' 1
f 2
f' 2

0

0

3

3

4

4

In [10]:
g 0 y = -y
g 1 y = y
g x y = x * y

g' x y =
  case (x, y) of
   (0,  y') -> -y'
   (1,  y') -> y'
   (x', y') -> x' * y'

In [11]:
let x = 1
 in
 let z = x + 2
  in (x,z)

(1,3)

In [12]:
(\x -> (\z -> (x,z)) (x+2)) 1

Line 1: Use tuple-section
Found:
\ z -> (x, z)
Why not:
(x,)

(1,3)

In [13]:
case 10 `mod` 2 of
  0 -> "even"
  1 -> "odd"

"even"

In [14]:
1 + 2 

3

In [15]:
let x = 1
 in
 let y = 2
  in x + y

3

In [16]:
-- let 과 where 는 비슷한 용도로 쓸 수 있다
f2 x =
  let y = 2
   in x + y

f3 x = x + y
  where
  y = 2

In [17]:
-- let은 값을 계산하는 계산식의 일부로 쓰일 수 있다
let y = 2
 in 1 + y

3

In [18]:
-- where는 새로운 함수/변수 정의에 따라붙는 지역변수 정의용
-- 그래서 이렇게 계산식의 일부로 사용할 수 없다 (let과 where의 차이)
1 + y
  where
  y = 2
-- 문법 오류

Parse error (line 4, column 3): parse error on input ‘where’

# hw3: 다음 인터프리터를 완성하라 (제출기한: 중간고사 전날 10/22일 밤; 6점)
인터프리터를 완성하고 팩토리알 프로그램을 Expr 문법으로 작성하여 eval로 실행시켜 보라.

작성한 각각의 문법요소에 대하여 최소 1개씩의 테스트 케이스를 함께 포함해서 제출해야 하며, 비교연산의 경우는 각각 최소 2개씩의 테스트 케이스를 포함해야 한다. 단, 팩토리알 프로그램에서 이미 활용한 요소들은 제외해도 무방하다. 예를 들어 아래에서 틀을 제시해준 팩토리알 예제를 eval로 실행할 때 이미 If와 Let을 활용하게 되므로 If와 Let에 대한 테스트 케이스는 별도로 작성하지 않아도 된다는 뜻이다.

배점은 다음과 같다.
 * eval 함수 작성 2점
 * 팩토리알 예제 성공적 작성 및 실행 2점
 * 각 요소에 대한 빠짐없는 테스트 케이스 2점 (하나라도 빠지면 1점 감점되며, 절반 이하 수준이면 0점)

eval 함수가 문법오류나 타입오류 등 제대로 받아들여지지 않는 정의일 경우는 무조건 전체 점수가 0점이다.

팩토리알 예제에 문법오류나 타입오류 등 제대로 받아들여지지 않는 정의이거나 받아들여진다 할지라도 실행 결과가 맞게 나오지 않으면 팩토리알 예제에 배정된 2점 배점을 전혀 받지 못한다. (즉 나머지가 완벽하더라도 최대 4점)

In [19]:
eval :: Expr -> Env -> Val
eval (I n) _ = VI n
eval (B b) _ = VB b
eval v@(Lam x e)         env = Cl v env
eval v@(Rec f (Lam x e)) env = Cl v env
eval (Var x)     env = lookup' x env
eval (App e1 e2) env =
  let v2 = eval e2 env in
  case eval e1 env of
    Cl v1@(Lam x e)         env1 -> eval e ((x,v2):env1)
    Cl v1@(Rec f (Lam x e)) env1 -> undefined
eval (If e e1 e2) env = undefined
eval (Let x e2 e) env = undefined -- let x=e2 in e 는 (\x.e) e2 의 문법설탕(syntactic sugar)
-- 산술연산
eval (Add e1 e2)  env = VI (x+y)
  where
  VI x = eval e1 env
  VI y = eval e2 env
eval (Mul e1 e2)  env = undefined
eval (Neg e)      env = undefined
-- 논리연산
eval (And e1 e2)  env = undefined
eval (Or e1 e2)   env = undefined
eval (Not e)      env = undefined
-- 비교연산
eval (Eq e1 e2)   env = undefined
eval (Ne e1 e2)   env = undefined
eval (Lt e1 e2)   env = undefined
eval (Gt e1 e2)   env = undefined
eval (Le e1 e2)   env = undefined
eval (Ge e1 e2)   env = undefined

In [26]:
eval (Add (I 3) (I 4)) []

eval (Add (I 3) (Var "x")) [("x",VI 5)]

-- 종이: f x
-- 하스켈로 정의한 문법구조: App f x

-- (\x.3+x) 5
eval (App (Lam "x"  (Add (I 3) (Var "x"))) (I 5)) []
-- let x=5 in 3+x
eval (Let "x" (I 5) (Add (I 3) (Var "x"))) []

VI 7

VI 8

VI 8

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:13:25 in interactive:Ghci405

In [52]:
-- eval을 완성한 후 factExpr를 완성하여 7팩토리알을 실행해 보라
factExpr = Rec undefined undefined
-- eval (Let "fact" factExpr (App (Var "fact") (I 7))) []
eval (App (Lam "fact" (App (Var "fact") (I 7))) factExpr) []

Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:13:25 in interactive:Ghci991

In [8]:
:type eval
:type uncurry eval

eval (App (Lam "x" (Var "x")) (Lam "x" (Var "x"))) []

sigma = [("twice",Cl (Lam "f"(Lam "x"(App f(App f x)))) [])]
      where
       f = Var "f"
       x = Var "x"

twice = Var "twice"

eval (App twice twice) sigma

eval :: Expr -> Env -> Val

uncurry eval :: (Expr, Env) -> Val

Cl (Lam "x" (Var "x")) []

Cl (Lam "x" (App (Var "f") (App (Var "f") (Var "x")))) [("f",Cl (Lam "f" (Lam "x" (App (Var "f") (App (Var "f") (Var "x"))))) [])]

In [9]:
import Data.List (intersperse)
dpEnv env = "{"++ concat (intersperse ", " [x++" ↦ "++dpVal v | (x,v)<-env]) ++ "}"

dpVal (VI n) = show n
dpVal (VB b) = show b
dpVal (Cl t env) = "⟨"++dpExpr t++", "++dpEnv env++"⟩"

dispEnv env = Display[html $ "<code>"++dpEnv env++"</code>"]
dispVal v = Display[html $ "<code>"++dpVal v++"</code>"]

In [10]:
dispEnv sigma

{twice ↦ ⟨λf.λx.f (f x), {}⟩}

In [11]:
putStrLn . dpVal $ eval (App twice twice) sigma
dispVal $ eval (App twice twice) sigma

⟨λx.f (f x), {f ↦ ⟨λf.λx.f (f x), {}⟩}⟩

⟨λx.f (f x), {f ↦ ⟨λf.λx.f (f x), {}⟩}⟩

In [12]:
idExpr

Lam "x" (Var "x")

In [13]:
putStrLn . dpVal $ eval (I 3) sigma
dispVal $ eval (I 3) sigma

doubleExpr = Lam "x" (Var "x" `Add` Var "x")

dispVal $ eval (App idExpr (I 3)) sigma
dispVal $ eval (App doubleExpr (I 3)) sigma

3

3

3

6

In [14]:
putStrLn . dpVal $ eval (B True) sigma
dispVal $ eval (B True) sigma

True

True